In [4]:
# Dataset: [road_width, traffic_volume, avg_speed]
data = [
    [10, 500, 60],
    [8, 1000, 40],
    [6, 1500, 20],
    [12, 400, 70],
    [7, 1200, 30],
    [5, 1800, 15]
]

# Split data into features (X) and target (y)
X = [row[:2] for row in data]  # Features: [road_width, traffic_volume]
y = [row[2] for row in data]  # Target: [avg_speed]

In [5]:
def normalize(data):
    min_val = min(data)
    max_val = max(data)
    return [(x - min_val) / (max_val - min_val) for x in data], min_val, max_val

# Normalize features and target
X_normalized = []
scales = []  # To store min-max scales for de-normalization
for i in range(len(X[0])):
    column = [row[i] for row in X]
    norm_col, min_val, max_val = normalize(column)
    X_normalized.append(norm_col)
    scales.append((min_val, max_val))

# Transpose normalized features
X_normalized = list(zip(*X_normalized))
y_normalized, y_min, y_max = normalize(y)


In [6]:
def linear_regression(X, y, epochs=1000, learning_rate=0.01):
    # Initialize weights and bias
    weights = [0.0] * len(X[0])  # One weight per feature
    bias = 0.0

    # Gradient Descent
    for epoch in range(epochs):
        total_error = 0
        for i in range(len(X)):
            # Predict using current weights and bias
            y_pred = sum([weights[j] * X[i][j] for j in range(len(X[0]))]) + bias
            error = y_pred - y[i]

            # Update weights and bias using gradient descent
            for j in range(len(weights)):
                weights[j] -= learning_rate * error * X[i][j]
            bias -= learning_rate * error

            # Accumulate error
            total_error += error ** 2
        
        # Print loss every 100 epochs
        if epoch % 100 == 0:
            print(f"Epoch {epoch}, Loss: {total_error / len(X):.4f}")

    return weights, bias


In [7]:
# Train the linear regression model
weights, bias = linear_regression(X_normalized, y_normalized)

print("Trained Weights:", weights)
print("Trained Bias:", bias)


Epoch 0, Loss: 0.3150
Epoch 100, Loss: 0.0086
Epoch 200, Loss: 0.0014
Epoch 300, Loss: 0.0011
Epoch 400, Loss: 0.0010
Epoch 500, Loss: 0.0010
Epoch 600, Loss: 0.0010
Epoch 700, Loss: 0.0010
Epoch 800, Loss: 0.0010
Epoch 900, Loss: 0.0010
Trained Weights: [0.6709427546164409, -0.3831057381730891]
Trained Bias: 0.33443083919946154


In [8]:
def predict(new_point, weights, bias, scales, y_min, y_max):
    # Normalize the input point using scales
    norm_point = [(new_point[i] - scales[i][0]) / (scales[i][1] - scales[i][0]) for i in range(len(new_point))]

    # Compute normalized prediction
    y_pred_norm = sum([weights[i] * norm_point[i] for i in range(len(weights))]) + bias

    # De-normalize the prediction
    y_pred = y_pred_norm * (y_max - y_min) + y_min
    return y_pred

# Test with a new data point
new_point = [9, 1100] 
predicted_speed = predict(new_point, weights, bias, scales, y_min, y_max)
print(f"Predicted Average Speed for {new_point}: {predicted_speed:.2f} km/h")


Predicted Average Speed for [9, 1100]: 43.95 km/h


In [9]:
def mean_squared_error(y_true, y_pred):
    return sum((yt - yp) ** 2 for yt, yp in zip(y_true, y_pred)) / len(y_true)


predictions = [predict(X[i], weights, bias, scales, y_min, y_max) for i in range(len(X))]
mse = mean_squared_error(y, predictions)
print(f"Mean Squared Error (MSE): {mse:.2f}")


Mean Squared Error (MSE): 3.07
